In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt
import statsmodels.api as sm

/home/sajinie/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
dengue_features_train = "dengue_features_train.csv"
dengue_labels_train = "dengue_labels_train.csv"

x_train = pd.read_csv(dengue_features_train, index_col=[0])
y_train = pd.read_csv(dengue_labels_train, index_col=[0])

# Seperate data for San Juan
sj_train_features = x_train.loc['sj']
sj_train_labels = y_train.loc['sj']
# sj_train_labels = sj_train_labels.drop(['year', 'weekofyear'], axis=1)

# Separate data for Iquitos
iq_train_features = x_train.loc['iq']
iq_train_labels = y_train.loc['iq']
iq_train_labels = iq_train_labels.drop(['year', 'weekofyear'], axis=1)

x_train.head()

,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
city,,,,,,,,,,,,,,,,,,,,,
sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [3]:
output = ['total_cases']
weather_station_measurements =  ['station_max_temp_c', 'station_min_temp_c', 'station_avg_temp_c',
     'station_precip_mm', 'station_diur_temp_rng_c']

satellite_precipitation_measurements = ['precipitation_amt_mm']

climate_forcast_measurements = ['reanalysis_sat_precip_amt_mm', 'reanalysis_dew_point_temp_k', 'reanalysis_air_temp_k', 
                          'reanalysis_relative_humidity_percent', 'reanalysis_specific_humidity_g_per_kg', 
                          'reanalysis_precip_amt_kg_per_m2', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 
                          'reanalysis_avg_temp_k', 'reanalysis_tdtr_k']

ndvi_columns = ['ndvi_se', 'ndvi_sw', 'ndvi_ne', 'ndvi_nw']

#data with all features and total cases for 'sj'. axis = 1 => by column
all_data_for_sj = pd.concat([sj_train_features, sj_train_labels], axis=1)
all_data_for_iq = pd.concat([iq_train_features, iq_train_labels], axis=1)

all_data_for_sj.head()

,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
city,,,,,,,,,,,,,,,,,,,,,
sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [4]:
from sklearn.preprocessing import Imputer

def fillMissingValues(x):

    # Drop irrelevant columns
    x = x.drop(['week_start_date'], axis=1)

    # Processing categoricals
    #One hot encoding categoricals
    x = pd.get_dummies(x)

    #without categoricals
    x = x.select_dtypes(exclude=['object'])
    
    x.fillna(method='ffill', inplace=True)

    return x

In [5]:
all_data = pd.concat([all_data_for_sj, all_data_for_iq], axis=0)
all_data.head(8)

,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
city,,,,,,,,,,,,,,,,,,,,,
sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6
sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,299.764286,...,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2
sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,299.221429,...,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4
sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,299.528571,...,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5


In [6]:
formatted_all_data = fillMissingValues(all_data)
formatted_all_data.head(8)

c = formatted_all_data['total_cases']
c.head()

#total_cases_excluded_formatted_data = formatted_all_data.drop(['total_cases'], axis=1)
#total_cases_excluded_formatted_data.head(8)

city
sj    4
sj    5
sj    4
sj    3
sj    6
Name: total_cases, dtype: int64

In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def rescaleData(y):
    x = pd.get_dummies(y)
    x = x.drop(['total_cases'], axis=1)
    array = x.values
    # separate array into input and output components
    X = array[:,0:21]
    scaler = MinMaxScaler(feature_range=(0, 1))
    rescaledX = scaler.fit_transform(X)
    
    # summarize transformed data
    np.set_printoptions(precision=3)
    print(rescaledX[0:5,:])
    
def rescaleDataNew(x):
    x = x.drop(['total_cases'], axis=1)
    # separate array into input and output components
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    x[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements] = scaler.fit_transform(x[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements])
    
#     x = scaler.fit_transform(x)
    # summarize transformed data
    np.set_printoptions(precision=3)
    return x
    
    
x = rescaleDataNew(formatted_all_data)
# print(x[0:5,:])
x.head()

    

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
city,,,,,,,,,,,,,,,,,,,,,
sj,1990,18,0.578226,0.614835,0.386418,0.395544,0.031797,0.388291,0.354667,0.314680,...,0.056091,0.381614,0.031797,0.262659,0.086660,0.430091,0.210393,0.174194,0.486239,0.029450
sj,1990,19,0.629943,0.657063,0.321190,0.359233,0.058423,0.472710,0.441778,0.489213,...,0.031446,0.479668,0.058423,0.418164,0.069133,0.565350,0.163498,0.322581,0.688073,0.015829
sj,1990,20,0.479441,0.690881,0.311879,0.384430,0.088428,0.548064,0.496000,0.657583,...,0.045749,0.594415,0.088428,0.586900,0.064265,0.565350,0.173638,0.354839,0.743119,0.076201
sj,1990,21,0.584823,0.770066,0.438912,0.491150,0.039324,0.575260,0.539556,0.643471,...,0.024365,0.552387,0.039324,0.566808,0.073028,0.645897,0.198986,0.425806,0.788991,0.007362
sj,1990,22,0.658698,0.788882,0.481601,0.509943,0.019252,0.645515,0.593778,0.701541,...,0.021385,0.555396,0.019252,0.628226,0.112950,0.802432,0.429658,0.535484,0.844037,0.010676


In [8]:
from sklearn.preprocessing import StandardScaler

def standardizeData(Y):
    X = pd.get_dummies(Y)
    X = X.drop(['total_cases'], axis=1)
    scaler = StandardScaler().fit(X[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements])
    X[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements] = scaler.transform(X[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements])
    # summarize transformed data
    np.set_printoptions(precision=3)
    return X

x = standardizeData(formatted_all_data)
# print(x[0:5,:])
x.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
city,,,,,,,,,,,,,,,,,,,,,
sj,1990,18,-0.062619,-0.203670,-0.055424,-0.291644,-0.760139,-0.826384,-1.171427,-1.854064,...,-0.187274,-1.230763,-0.760139,-1.772476,-0.641063,-1.356941,-0.558712,-1.552648,-1.338708,-0.489876
sj,1990,19,0.278949,0.118034,-0.541096,-0.556484,-0.522502,-0.357358,-0.616859,-0.847423,...,-0.511878,-0.671592,-0.522502,-0.890359,-0.713680,-0.363964,-0.807790,-0.379236,0.063840,-0.646168
sj,1990,20,-0.715063,0.375662,-0.610427,-0.372705,-0.254704,0.061303,-0.271668,0.123667,...,-0.323488,-0.017231,-0.254704,0.066811,-0.733851,-0.363964,-0.753935,-0.124146,0.446353,0.046589
sj,1990,21,-0.019050,0.978910,0.335437,0.405659,-0.692961,0.212399,0.005616,0.042275,...,-0.605150,-0.256904,-0.692961,-0.047160,-0.697543,0.227360,-0.619298,0.437051,0.765113,-0.743323
sj,1990,22,0.468869,1.122261,0.653287,0.542732,-0.872102,0.602730,0.350807,0.377199,...,-0.644398,-0.239741,-0.872102,0.301239,-0.532139,1.376536,0.605899,1.304356,1.147626,-0.705306


In [9]:
from sklearn.preprocessing import Normalizer

def normalizeData(y):
    x = pd.get_dummies(y)
    x = x.drop(['total_cases'], axis=1)
    scaler = Normalizer().fit(x)
    x[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements] = scaler.transform(x[ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements])
    
    return x

# ndvi_columns + satellite_precipitation_measurements + weather_station_measurements + climate_forcast_measurements


def n(x):
    x = x.drop(['total_cases','year','weekofyear'], axis=1)
    array = x.values
    # separate array into input and output components
    X = array[:,0:21]
    scaler = Normalizer().fit(X)
    normalizedX = scaler.transform(X)
    # summarize transformed data
    np.set_printoptions(precision=3)
    print(normalizedX[0:5,:])



x = normalizeData(formatted_all_data)
x.head()
# print(x[0:5,:])

# n(formatted_all_data)



,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
city,,,,,,,,,,,,,,,,,,,,,
sj,1990,18,0.000183,0.000155,0.000296,0.000265,0.018530,0.443952,0.444206,0.436256,...,0.047741,0.109455,0.018530,0.020906,0.003922,0.037958,0.010294,0.043862,0.029838,0.023871
sj,1990,19,0.000253,0.000211,0.000241,0.000231,0.033916,0.443212,0.443555,0.436880,...,0.026663,0.114988,0.033916,0.022848,0.003524,0.039704,0.009469,0.047114,0.032994,0.012782
sj,1990,20,0.000048,0.000255,0.000232,0.000252,0.050999,0.441161,0.441305,0.436219,...,0.038538,0.121154,0.050999,0.024877,0.003396,0.039445,0.009576,0.047544,0.033665,0.061129
sj,1990,21,0.000191,0.000363,0.000337,0.000350,0.022770,0.443218,0.443576,0.437767,...,0.020605,0.119092,0.022770,0.024716,0.003600,0.040724,0.010038,0.049364,0.034540,0.005930
sj,1990,22,0.000290,0.000388,0.000372,0.000366,0.011130,0.443322,0.443538,0.437850,...,0.018057,0.119090,0.011130,0.025473,0.004461,0.042839,0.013871,0.051804,0.035375,0.008585


In [10]:
n(formatted_all_data)

[[1.829e-04 1.547e-04 2.961e-04 2.650e-04 1.853e-02 4.440e-01 4.442e-01
  4.363e-01 4.473e-01 4.415e-01 4.774e-02 1.095e-01 1.853e-02 2.091e-02
  3.922e-03 3.796e-02 1.029e-02 4.386e-02 2.984e-02 2.387e-02]
 [2.525e-04 2.113e-04 2.413e-04 2.311e-04 3.392e-02 4.432e-01 4.436e-01
  4.369e-01 4.472e-01 4.405e-01 2.666e-02 1.150e-01 3.392e-02 2.285e-02
  3.524e-03 3.970e-02 9.469e-03 4.711e-02 3.299e-02 1.278e-02]
 [4.762e-05 2.554e-04 2.321e-04 2.523e-04 5.100e-02 4.412e-01 4.413e-01
  4.362e-01 4.437e-01 4.390e-01 3.854e-02 1.212e-01 5.100e-02 2.488e-02
  3.396e-03 3.944e-02 9.576e-03 4.754e-02 3.366e-02 6.113e-02]
 [1.907e-04 3.633e-04 3.373e-04 3.497e-04 2.277e-02 4.432e-01 4.436e-01
  4.378e-01 4.468e-01 4.403e-01 2.061e-02 1.191e-01 2.277e-02 2.472e-02
  3.600e-03 4.072e-02 1.004e-02 4.936e-02 3.454e-02 5.930e-03]
 [2.904e-04 3.881e-04 3.718e-04 3.661e-04 1.113e-02 4.433e-01 4.435e-01
  4.378e-01 4.468e-01 4.403e-01 1.806e-02 1.191e-01 1.113e-02 2.547e-02
  4.461e-03 4.284e-02 1.387e

In [11]:
def lagData(x, shift):

#     i= formatted_all_data.iloc[index:]
#     j= formatted_all_data.precipitation_amt_mm

    j = x['precipitation_amt_mm'][0]

    x['pre_precipitation_amt_mm'] = x['precipitation_amt_mm'].shift(shift)
    x['pre_precipitation_amt_mm'][:shift] = j
    return x
    
x = lagData(formatted_all_data, 5)
print (x[['weekofyear','pre_precipitation_amt_mm','precipitation_amt_mm']])

      weekofyear  pre_precipitation_amt_mm  precipitation_amt_mm
city                                                            
sj            18                     12.42                 12.42
sj            19                     12.42                 22.82
sj            20                     12.42                 34.54
sj            21                     12.42                 15.36
sj            22                     12.42                  7.52
sj            23                     12.42                  9.58
sj            24                     22.82                  3.48
sj            25                     34.54                151.12
sj            26                     15.36                 19.32
sj            27                      7.52                 14.41
sj            28                      9.58                 22.27
sj            29                      3.48                 59.17
sj            30                    151.12                 16.48
sj            31         

/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
import pandas as pd 
import numpy as np
df = pd.DataFrame({"date": ["2000-01-03", "2000-01-03", "2000-03-05", "2000-01-03", "2000-03-05",
                        "2000-03-05", "2000-07-03", "2000-01-03", "2000-07-03", "2000-07-03"],
               "variable": ["A", "A", "A", "B", "B", "B", "C", "C", "C", "D"],
               "no": [1, 2.2, 3.5, 1.5, 1.5, 1.2, 1.3, 1.1, 2, 3],
               "value": [0.469112, -0.282863, -1.509059, -1.135632, 1.212112, -0.173215,
                         0.119209, -1.044236, -0.861849, None]})

In [13]:
df['value']

0    0.469112
1   -0.282863
2   -1.509059
3   -1.135632
4    1.212112
5   -0.173215
6    0.119209
7   -1.044236
8   -0.861849
9         NaN
Name: value, dtype: float64

In [14]:
df['value'].shift(1)

0         NaN
1    0.469112
2   -0.282863
3   -1.509059
4   -1.135632
5    1.212112
6   -0.173215
7    0.119209
8   -1.044236
9   -0.861849
Name: value, dtype: float64

In [112]:
def pre_process_sj(x):
    
    features = ['weekofyear',
                'reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c',
               'precipitation_amt_mm']
    x = x[features]
    
#     x['week_start_date'] = pd.to_datetime(x['week_start_date'])
#     for i in range(1,5):
#         x['quarter_' + str(i)] = x['week_start_date'].apply(lambda date: 1 if (
#             ((i-1)*3<date.to_datetime().month) and (date.to_datetime().month <= i * 3)) else 0)
        
    x['quarter'] = x['weekofyear'].apply(lambda week: 1 if((week-1)%13==0) 
                                         else (2 if ((week-1)%13==1) 
                                               else(3 if ((week-1)%13==2) else 4)))
#     x = x.drop(['week_start_date','quarter_1','quarter_3',],axis=1)
#precipitation_amt_mm of week before last week
    i= x.iloc[-1:]
    j= x.precipitation_amt_mm
    j = j[:-1]
    col = []
    for k in i.precipitation_amt_mm:
        col.append(k)
    for k in j:
        col.append(k)
    x['pre_precipitation_amt_mm'] = col
    
    
    # fill missing values
    x.fillna(method='ffill', inplace=True)
    return x


def pre_process_iq(x):
    
    features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c']
    features = ['weekofyear',
                'reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c',
               'precipitation_amt_mm']
    x = x[features]

    # fill missing values
    x.fillna(method='ffill', inplace=True)

    # add labels to dataframe
    return x

In [113]:
sj_train_features_formated = pre_process_sj(sj_train_features)

iq_train_features_formated = pre_process_iq(iq_train_features)


/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sajinie/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

In [17]:
sj_train_features_formated.head()

,weekofyear,reanalysis_specific_humidity_g_per_kg,reanalysis_dew_point_temp_k,station_avg_temp_c,precipitation_amt_mm,quarter,pre_precipitation_amt_mm
city,,,,,,,
sj,18,14.012857,292.414286,25.442857,12.42,4,0.00
sj,19,15.372857,293.951429,26.714286,22.82,4,12.42
sj,20,16.848571,295.434286,26.714286,34.54,4,22.82
sj,21,16.672857,295.310000,27.471429,15.36,4,34.54
sj,22,17.210000,295.821429,28.942857,7.52,4,15.36


In [61]:
def normalizeDataTest1(y):
    x = pd.get_dummies(y)
    features = ['weekofyear',
                'reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c',
                'quarter',
                'pre_precipitation_amt_mm',
               'precipitation_amt_mm']
    x = x[features]
    scaler = Normalizer().fit(x)
    x[features] = scaler.transform(x[features])
    
    return x

def standardizeDataTest1(Y):
    x = pd.get_dummies(Y)
    features = ['weekofyear',
                'reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c',
                'quarter',
                'pre_precipitation_amt_mm',
               'precipitation_amt_mm']
    x = x[features]
    scaler = StandardScaler().fit(x[features])
    x[features] = scaler.transform(x[features])
    # summarize transformed data
    np.set_printoptions(precision=3)
    return x



#'weekofyear','reanalysis_specific_humidity_g_per_kg','reanalysis_dew_point_temp_k','station_avg_temp_c', 'precipitation_amt_mm','quarter', 'pre_precipitation_amt_mm'

In [62]:
def normalizeDataIq(y):
    x = pd.get_dummies(y)
    features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c']
    
    x = x[features]
    scaler = Normalizer().fit(x)
    x[features] = scaler.transform(x[features])
    
    return x

def standardizeDataIq(Y):
    x = pd.get_dummies(Y)
    features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c']
    x = x[features]
    scaler = StandardScaler().fit(x[features])
    x[features] = scaler.transform(x[features])
    # summarize transformed data
    np.set_printoptions(precision=3)
    return x



In [63]:
sj_train_features_formated_new = normalizeDataTest1(sj_train_features_formated)
sj_train_features_formated_new.head()

,weekofyear,reanalysis_specific_humidity_g_per_kg,reanalysis_dew_point_temp_k,station_avg_temp_c,quarter,pre_precipitation_amt_mm,precipitation_amt_mm
city,,,,,,,
sj,0.061080,0.047551,0.992267,0.086337,0.013573,0.000000,0.042146
sj,0.063901,0.051702,0.988613,0.089845,0.013453,0.041771,0.076748
sj,0.066516,0.056035,0.982552,0.088846,0.013303,0.075895,0.114873
sj,0.069951,0.055537,0.983675,0.091507,0.013324,0.115052,0.051164
sj,0.073564,0.057547,0.989172,0.096780,0.013375,0.051361,0.025145


In [64]:
x = standardizeDataTest1(sj_train_features_formated)

y = standardizeDataIq(iq_train_features_formated)
# print(x[0:5,:])
x.head()

,weekofyear,reanalysis_specific_humidity_g_per_kg,reanalysis_dew_point_temp_k,station_avg_temp_c,quarter,pre_precipitation_amt_mm,precipitation_amt_mm
city,,,,,,,
sj,-0.566356,-1.624972,-1.714497,-1.100409,0.501176,-0.791293,-0.513013
sj,-0.499751,-0.753081,-0.734948,-0.201443,0.501176,-0.513013,-0.279993
sj,-0.433146,0.192993,0.210006,-0.201443,0.501176,-0.279993,-0.017398
sj,-0.366541,0.080343,0.130805,0.333897,0.501176,-0.017398,-0.447140
sj,-0.299936,0.424704,0.456714,1.374273,0.501176,-0.447140,-0.622801


In [137]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
def train(x_formated,y_train):
    # split data into training and validation data, for both predictors and target
    # The split is based on a random number generator. Supplying a numeric value to
    # the random_state argument guarantees we get the same split every time we
    # run this script.
    x, val_X, y, val_y = train_test_split(x_formated, y_train, test_size=0.1, random_state=0, shuffle=False)
    total_y = y[['total_cases'] ]
    total_y.fillna(method='ffill', inplace=True)
#     model = RandomForestRegressor(max_features=2,max_depth=4,warm_start=True, min_samples_split=4,criterion='mae', n_estimators=180, min_samples_leaf=2)
    model = RandomForestRegressor(n_estimators=150,criterion='mae',max_depth=7,warm_start=True,)
    model.fit(x, total_y)
    pred_val = model.predict(val_X)
    print("MAE : " + str(mean_absolute_error(val_y['total_cases'], pred_val)))
    importances = model.feature_importances_
    print (x.columns)
    print (importances)
    return model



In [138]:
forest_model_sj = train(sj_train_features_formated,sj_train_labels)

/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


MAE : 19.559929078014182
Index(['weekofyear', 'reanalysis_specific_humidity_g_per_kg',
       'reanalysis_dew_point_temp_k', 'station_avg_temp_c',
       'precipitation_amt_mm', 'quarter', 'pre_precipitation_amt_mm'],
      dtype='object')
[0.378 0.146 0.126 0.143 0.082 0.019 0.105]


In [139]:
forest_model_iq = train(iq_train_features_formated,iq_train_labels)

/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


MAE : 4.954615384615384
Index(['weekofyear', 'reanalysis_specific_humidity_g_per_kg',
       'reanalysis_dew_point_temp_k', 'station_avg_temp_c',
       'precipitation_amt_mm'],
      dtype='object')
[0.196 0.18  0.213 0.227 0.185]


In [140]:
dengue_features_test = "dengue_features_test.csv"
x_test = pd.read_csv(dengue_features_test, index_col=[0])

sj_test_features = x_test.loc['sj']
iq_test_features = x_test.loc['iq']

# x_test.head()

sj_test_formated = pre_process_sj(sj_test_features)
iq_test_formated = pre_process_iq(iq_test_features)

sj_test_features.head()

pred_sj = pd.DataFrame(forest_model_sj.predict(sj_test_formated))
pred_sj = pred_sj.round().astype(int)
pred_sj.columns = ['total_cases']

# output_sj = pd.concat([sj_test_features[['city','year', 'weekofyear']], pred_sj], axis=1)

pred_iq = pd.DataFrame(forest_model_iq.predict(iq_test_formated))
pred_iq = pred_iq.round().astype(int)
pred_iq.columns = ['total_cases']


/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/sajinie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sajinie/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

In [141]:
submission = pd.read_csv("submission_format.csv")
print (submission.shape)
print (pred_sj.shape)
print (pred_iq.shape)
submission.total_cases = np.concatenate([pred_sj, pred_iq])
submission.to_csv("submission_test-1.csv",index=False)

(416, 4)
(260, 1)
(156, 1)
